Notebook to Extract Cananda Neighbourhood Data from Wikipedia

In [20]:
#Let us import required libraries
from pandas.io.html import read_html

In [21]:
#Let us read the Wikipedia Page
page = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wikitables = read_html(page, attrs={"class":"wikitable"})
#Let us see number of tables we have generated. We need to ensure that we use correct one
print ("Extracted {num} wikitables".format(num=len(wikitables)))

Extracted 1 wikitables


Good that we have only one wikitable. Let us see if this is the one we are looking for

In [22]:
#Print the retrieved wikitable
print (wikitables[0].head())

          0             1                 2
0  Postcode       Borough     Neighbourhood
1       M1A  Not assigned      Not assigned
2       M2A  Not assigned      Not assigned
3       M3A    North York         Parkwoods
4       M4A    North York  Victoria Village


Yes, this looks to be the one we are interested in. Let us proceed.

In [23]:
#For the sake of simplicity let us rename the table
canada_neighbourhood_details_df = wikitables[0]
#We need to change column headers to "Postcode","Borough" and "Neighbourhood"
canada_neighbourhood_details_df.columns = canada_neighbourhood_details_df.iloc[0]
#Let us drop the row which we have used as column header and also reindex the table
canada_neighbourhood_details_df = canada_neighbourhood_details_df.reindex(canada_neighbourhood_details_df.index.drop(0))
print (canada_neighbourhood_details_df.head())

0 Postcode           Borough     Neighbourhood
1      M1A      Not assigned      Not assigned
2      M2A      Not assigned      Not assigned
3      M3A        North York         Parkwoods
4      M4A        North York  Victoria Village
5      M5A  Downtown Toronto      Harbourfront


The Table has been Extracted Successfully. Let us now perform required cleanup

In [24]:
#Let us create new dataframe by removing all "Not assigned" values in "Borough"
mod_can_nei_det_df = canada_neighbourhood_details_df[canada_neighbourhood_details_df.Borough != "Not assigned"]
mod_can_nei_det_df = mod_can_nei_det_df.reset_index(drop=True)
print (mod_can_nei_det_df.head())

0 Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights


Now that we have reset the index and removed the "Not assigned" values in Borough column. Let us see if we have any "Not assigned" values in "Neighbourhood".

In [25]:
print (mod_can_nei_det_df[mod_can_nei_det_df.Neighbourhood == "Not assigned"].head(10))

0 Postcode       Borough Neighbourhood
6      M7A  Queen's Park  Not assigned


Luckily, there is only one such value. We can directly modify this value. Otherwise we would have to write a loop to change all the values. We need to set the "Neighbourhood" value equal to "Borough" value in this case.

In [26]:
#Modification of Neighbourhood value
mod_can_nei_det_df.iloc[[6], [2]] = mod_can_nei_det_df.iloc[[6], [1]].values
#Let us see if the above step has worked
print ("Value Post assignment is = \n{}".format(mod_can_nei_det_df.iloc[[6], [2]]))
print ("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print (mod_can_nei_det_df[mod_can_nei_det_df.Neighbourhood == "Not assigned"].head(10))
print ("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
print ("Shape of the table after above modifications is {}".format(mod_can_nei_det_df.shape))

Value Post assignment is = 
0 Neighbourhood
6  Queen's Park
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Empty DataFrame
Columns: [Postcode, Borough, Neighbourhood]
Index: []
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Shape of the table after above modifications is (212, 3)


Good Enough. Now the last bit. We need to consolidate all the Neighbourhood's having same Postcode. We can use groupby function for the same.

In [27]:
mod_can_nei_det_df = mod_can_nei_det_df.groupby(['Postcode','Borough'],as_index=False).agg(','.join)

In [28]:
#Let us see if we have our required form
print (mod_can_nei_det_df.head())

0 Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae


Neat! Let us check the shape of the table now.

In [29]:
#Print the shape of the table
print ("Shape of table after grouping Neighbourhood values is {}".format(mod_can_nei_det_df.shape))

Shape of table after grouping Neighbourhood values is (103, 3)
